# Cohort File Exploration

In [ ]:
import os

import pandas as pd
from pyarrow import parquet as pq

# Initial configuration

This notebook assumes that the raw eICU `.csv` files are available in the `data/raw/eICU` directory and that the cohort `.parquet` files are available in `data/cohorts/sepsis_eicu_robin`.

In [ ]:
path_to_raw_eicu_data = "../data/raw/eICU"
path_to_cohorts = "../data/cohorts/sepsis_eicu_robin"

# Investigate what is inside of each file

## Basic inspection

In [ ]:
for filename in os.listdir(path_to_cohorts):
    print ("************************************************")
    print (f"Loading file {filename}")
    print ("************************************************")
    filepath = os.path.join(path_to_cohorts, filename)

    if 'parquet' in filename:
        data = pq.read_table(filepath).to_pandas()
        
        print (f"** The shape of the data in this file is: {data.shape}", end="\n\n")
        print (f"** The columns in this file are: {data.columns}", end="\n\n")
        print (f"** The number of unique stays in this file is: {len(data.stay_id.unique())}", end="\n\n")
        
        print (data.info(), end="\n\n")
        print (data.describe(), end="\n\n")
        
        if 'label' in data.columns:
            instance_percentage_sepsis = data[data['label']==True].shape[0]/data.shape[0]
            
            visit_labels = data.groupby(by="stay_id").max()['label']
            instance_visits_sepsis = visit_labels[visit_labels==True].shape[0]/visit_labels.shape[0]
            
            print (f"** The proportion of all clinical concept readings marked with a positive sepsis label is: {instance_percentage_sepsis*100.0:.3f}%")
            print (f"** The proportion of all stays that at some point have a positive sepsis label is: {instance_visits_sepsis*100.0:.3f}%")

    print ("************************************************")
    print ("************************************************\n\n")
        

# See whether the same stay_ids can be found in the original (raw) eICU data

## Load the raw data and the static data for the cohort

In [ ]:
eICU_raw_patient_data = pd.read_csv(os.path.join(path_to_raw_eicu_data, "patient.csv"))
eICU_cohort_concept_data = pq.read_table(os.path.join(path_to_cohorts, "sta.parquet")).to_pandas()

In [ ]:
print (f"The shape of the raw eICU patient table is: {eICU_raw_patient_data.shape}")
print (f"The shape of the cohort concept data is: {eICU_cohort_concept_data.shape}")

print ("\n")

print (f"The number of unique patient stays in the raw eICU patient table is: {len(eICU_raw_patient_data['patientunitstayid'].unique())}")

## Join the two tables on `stay_id` and check the resulting shapes

In [ ]:
merged_data = pd.merge(left = eICU_raw_patient_data, right=eICU_cohort_concept_data, left_on="patientunitstayid", right_on = "stay_id", how="inner")

print (f"The shape of the raw eICU patient table is: {eICU_raw_patient_data.shape}")
print (f"The shape of the cohort concept data is: {eICU_cohort_concept_data.shape}")
print (f"The shape of the merged data is {merged_data.shape}")

The fact that there are as many rows after merging as there were in the cohort concept data, when merging using an `inner` join, tells us that the operation was able to identify all `stay_id`s from the cohort concept data, in the raw eICU patient table. 

However, that does not yet mean that the stay_ids generated with yaib-cohorts are truly consistent.

## Subtract the ages (that come from the two different tables) for each row in the merged data. If the difference is always 0, then the ages in the two tables are the same for each stay_id.

In [ ]:
merged_data

In [ ]:
# the age_x column (comming from the raw eICU patient column) is of type str with a value '> 89' which we need to replace with something (in our case '90') to cast it to a float type
age_diff = merged_data['age_x'].replace('> 89', '90').astype(float) - merged_data['age_y'].astype(float)

print (f"The ages from both columns match for {(age_diff == 0.0).sum()} out of {age_diff.shape[0]} stay_ids")

## What are the values of the two columns when they don't match?

In [ ]:
merged_data[['stay_id', 'patientunitstayid', 'age_x', 'age_y']].loc[age_diff != 0.0, :]

## Do the same for the hights and weights to confirm

In [ ]:
height_diff = merged_data['admissionheight'].astype(float) - merged_data['height'].astype(float)
weight_diff = merged_data['admissionweight'].astype(float) - merged_data['weight'].astype(float)

print (f"The heights from both columns match for {(height_diff == 0.0).sum()} out of {height_diff.shape[0]} stay_ids")
print (f"The weights from both columns match for {(weight_diff == 0.0).sum()} out of {weight_diff.shape[0]} stay_ids")

These are the same numbers as the number of non NaN values in each of these columns

# Conclusion

The fact that we are able to find the `stay_id`s from the cohort concept data in the original eICU patient table, coupled with the fact that the ages, heights and weights of these patients are the same in all cases when these values aren't missing makes me believe that the `stay_id`s can in fact be matched 1:1 to those in the original eICU data.